# 3. A ChatGPT Conversation with PyLapi

There are times you want to have more control over API requests and responses, keep track of the API call sequence, or simply monitor the API interactions.
These tasks can be tedious and time-consuming. Your code may become more complex and therefore less readable.

Here is a typical use case: In the OpenAI API, each ChatGPT interaction is independent.
If you want to keep the conversation flowing, you need to keep track of the context.

For example, you ask ChatGPT, "Where is the Sydney Opera House?" After receiving the answer, you ask a follow-up question: "How do I get *there*?"
To allow ChatGPT to work out that "there" means "the Sydney Opera House", you need to include the entire conversation in each API call.

## Keep the chat rolling

The `PyLapi` resource below maintains a ChatGPT conversation flow by saving all questions and answers in the `Conversation` resource class, so whenever its `ask()` method is called, the entire conversation is sent to the OpenAI API, so its response will be context-aware.

All is handcrafted in just 39 lines, including space lines, without any automation or input from the OpenAPI specification, thanks to the rich features inherited from `PyLapi`.

In [1]:
from pylapi import PyLapi

class oAPI(PyLapi):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.api_url = "https://api.openai.com/v1"

@oAPI.resource_class("conversation")
class Conversation(oAPI):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.message_bank = []

    @oAPI.resource_method("/chat/completions", "POST", load="$")
    def ask(self, question: str):

        @oAPI.callback
        def request(self, role="user", **kwargs):
            model_id = kwargs["model"] if "model" in kwargs else "gpt-3.5-turbo"
            self.message_bank.append({
                "role": role,
                "content": kwargs["question"],
            })
            self.raw_request["json"] = {
                "model": model_id,
                "messages": self.message_bank,
                "temperature": 0.7
            }
            self.raw_request["params"] = {}

        @oAPI.callback
        def response(self, **kwargs):
            if self.response_ok() and "choices" in self.response_data:
                choices = self.response_data["choices"]
                self.message_bank.append(choices[0]["message"])
                answers = [_["message"]["content"] for _ in choices]
                self.response_data.update({
                    "answers": answers,
                })

The top-level API class `oAPI` inherits from `PyLapi` and sets the API URL `self.api_url`. The `Conversation` resource class inherits from `oAPI` and is decorated by `@oAPI.resource_class` with resource name set to `"conversation"`.
The `__init__()` method creates and initialises the internal message bank.

The `@oAPI.resource_method` decorator makes the `ask()` method `POST` to the API route `/chat/completions`, and `load` the entire response (as denoted by `$`) into the resource `data` attribute.

There are two `callback` methods:
- `request()` adds the user's question to the message bank before PyLapi sends it to ChatGPT.
- `response()` adds ChatGPT's answers to the message bank and also creates the "answers" data attribute.

---
Before trying it out, you need to [sign up with OpenAI](https://platform.openai.com/signup), obtain your [API key](https://platform.openai.com/account/api-keys), and save it to `._osecret` under the current directory for authenticating the API.

IMPORTANT: Please store your API key securely without exposing it to any output or repositories.

In [2]:
# Authenticate with your OpanAI API key saved in `._osecret`.
oAPI.auth(open("._osecret", "r").readlines()[0].strip())
conversation = oAPI.resource("conversation")

## Let's talk!

Now you may ask your first question. Feel free to change the question to one that interests you.

In [3]:
conversation.ask("Where is the Sydney Opera House?")
print(conversation.data.answers[0])

The Sydney Opera House is located in Sydney, Australia.


When you ask a follow-up question, the `conversation` resource will send the entire conversation from the beginning to ChatGPT so it is aware of the context.

In [4]:
conversation.ask("How to get there?")
print(conversation.data.answers[0])

There are several ways to get to the Sydney Opera House. Here are a few options:

1. By public transport: You can take a train to Circular Quay station, which is a short walk from the Opera House. Circular Quay is a major transport hub in Sydney, and you can easily catch a train, bus, or ferry to get there.

2. By ferry: Sydney Opera House is situated right on the harbor, so taking a ferry is a scenic way to reach it. Ferries operate regularly from Circular Quay, and there are various routes available.

3. By bus: There are numerous bus services that stop near the Opera House. You can check the local bus schedule and find a route that suits your location.

4. By car: If you prefer to drive, there are parking facilities available at the Opera House and nearby areas. However, parking can be limited and expensive, so it's worth considering other transportation options.

It's a good idea to plan your journey in advance, considering factors like traffic, availability of public transport, an

You may also give intructions to ChatGPT using the "system" role.

In [5]:
conversation.ask("Speak like Harry Potter", role="system")
print(conversation.data.answers[0])

To reach the Sydney Opera House, one must embark on a magical journey! Hop on a trusty broomstick and soar through the skies, guided by the stars. Fly over the enchanting city of Sydney until you reach the sparkling waters of the harbor. Descend gracefully near Circular Quay, where the Opera House awaits. Alternatively, you can apparate directly to Circular Quay, but be mindful of the Muggles around you. Remember, the Opera House is a bewitching sight, so take a moment to appreciate its grandeur before you indulge in the wonders it holds within.


You may print the message bank at any time to review the conversation so far.

In [6]:
import json
print(json.dumps(conversation.message_bank, indent=2))

[
  {
    "role": "user",
    "content": "Where is the Sydney Opera House?"
  },
  {
    "role": "assistant",
    "content": "The Sydney Opera House is located in Sydney, Australia."
  },
  {
    "role": "user",
    "content": "How to get there?"
  },
  {
    "role": "assistant",
    "content": "There are several ways to get to the Sydney Opera House. Here are a few options:\n\n1. By public transport: You can take a train to Circular Quay station, which is a short walk from the Opera House. Circular Quay is a major transport hub in Sydney, and you can easily catch a train, bus, or ferry to get there.\n\n2. By ferry: Sydney Opera House is situated right on the harbor, so taking a ferry is a scenic way to reach it. Ferries operate regularly from Circular Quay, and there are various routes available.\n\n3. By bus: There are numerous bus services that stop near the Opera House. You can check the local bus schedule and find a route that suits your location.\n\n4. By car: If you prefer to dr

## What's the point?

Beyond code simplification, the `PyLapi` resource model and callback features blur the line between frontend custom functions and backend API methods.

In the above example, the application calls the `conversation.ask()` method "naturally", without even knowing that it is not a native API method.

This can be very useful to emulate new API methods so your frontend developers can start using them even before they are released in the native API.

Moreover, there are functionalities that can be provided at the frontend easily but are very complicated to implement at the backend.

In the above example, keeping the context at the backend involves passing handles (e.g., cookies) to and fro, storing the context in persistent storage, protecting the data in transit and at rest, etc.

---
In the next tutorial, we are going to explore more advanced features of `PyLapi` and see how it can make a programmer's life easier.

## End of page